# Панорамная сшивка изображений

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import time
import os

In [ ]:
# ===============================
# Загрузка или создание тестовых изображений
# ===============================

def create_panorama_test_images():
    """Создаёт набор перекрывающихся изображений для сшивки"""

    print("📝 Создаём синтетические изображения для панорамы...")

    # Базовое широкое изображение
    base_width = 1200
    base_height = 400
    base = np.ones((base_height, base_width, 3), dtype=np.uint8) * 200

    # Добавляем фон (градиент неба)
    for y in range(base_height):
        intensity = int(180 - y * 0.2)
        base[y, :] = [intensity, intensity + 20, intensity + 40]

    # Добавляем объекты в разных позициях
    # Здания
    buildings = [
        (100, 200, 150, 350, (120, 100, 100)),
        (200, 180, 250, 350, (100, 110, 120)),
        (400, 160, 480, 350, (110, 100, 100)),
        (550, 190, 620, 350, (100, 105, 115)),
        (750, 170, 830, 350, (115, 105, 105)),
        (900, 185, 970, 350, (105, 110, 120)),
        (1050, 195, 1120, 350, (110, 100, 110))
    ]

    for x1, y1, x2, y2, color in buildings:
        cv2.rectangle(base, (x1, y1), (x2, y2), color, -1)
        cv2.rectangle(base, (x1, y1), (x2, y2), (80, 80, 80), 2)

        # Окна
        for wy in range(y1 + 20, y2 - 20, 30):
            for wx in range(x1 + 10, x2 - 10, 25):
                cv2.rectangle(base, (wx, wy), (wx + 15, wy + 20), (200, 200, 150), -1)

    # Деревья (зелёные треугольники)
    trees = [(300, 320), (650, 310), (850, 305)]
    for tx, ty in trees:
        tree = np.array([[tx, ty - 50], [tx - 30, ty], [tx + 30, ty]], np.int32)
        cv2.fillPoly(base, [tree], (80, 150, 80))
        cv2.rectangle(base, (tx - 8, ty), (tx + 8, ty + 30), (100, 80, 60), -1)

    # Добавляем текстуру (шум)
    noise = np.random.randint(-15, 15, base.shape, dtype=np.int16)
    base = np.clip(base.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    # Создаём 3 перекрывающихся изображения
    images = []

    # Левое изображение
    img1 = base[:, 0:500].copy()
    images.append(img1)

    # Центральное изображение (перекрытие ~30%)
    img2 = base[:, 350:850].copy()
    # Добавляем небольшое изменение яркости
    img2 = np.clip(img2.astype(np.int16) + 10, 0, 255).astype(np.uint8)
    images.append(img2)

    # Правое изображение
    img3 = base[:, 700:1200].copy()
    # Добавляем небольшое изменение яркости
    img3 = np.clip(img3.astype(np.int16) - 10, 0, 255).astype(np.uint8)
    images.append(img3)

    print(f"✅ Создано {len(images)} изображений для панорамы")
    return images

# Загружаем или создаём изображения
images = create_panorama_test_images()

# Визуализируем исходные изображения
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, img in enumerate(images):
    axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[i].set_title(f'Изображение {i+1}\nРазмер: {img.shape[1]}x{img.shape[0]}')
    axes[i].axis('off')

plt.suptitle('Исходные изображения для панорамы (с перекрытием)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 1: Детекция и сопоставление ключевых точек
# ===============================

print("\n📋 ЗАДАНИЕ 1: Детекция и сопоставление ключевых точек")

# Инициализируем детектор
try:
    detector = cv2.SIFT_create()
    print("✅ Используем SIFT")
except:
    detector = cv2.ORB_create(nfeatures=2000)
    print("✅ Используем ORB")

# Детектируем ключевые точки и дескрипторы для всех изображений
keypoints_list = []
descriptors_list = []

for i, img in enumerate(images):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, des = detector.detectAndCompute(gray, None)
    keypoints_list.append(kp)
    descriptors_list.append(des)
    print(f"Изображение {i+1}: {len(kp)} ключевых точек")

# Визуализируем ключевые точки
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (img, kp) in enumerate(zip(images, keypoints_list)):
    img_with_kp = cv2.drawKeypoints(img, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    axes[i].imshow(cv2.cvtColor(img_with_kp, cv2.COLOR_BGR2RGB))
    axes[i].set_title(f'Изображение {i+1}: {len(kp)} точек')
    axes[i].axis('off')

plt.suptitle('Ключевые точки на всех изображениях', fontsize=14)
plt.tight_layout()
plt.show()

# Сопоставляем соседние изображения
def match_images(des1, des2, ratio=0.75):
    """Сопоставляет дескрипторы двух изображений"""

    # Определяем тип дескриптора
    if des1.dtype == np.float32:
        matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    else:
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

    # Находим 2 ближайших соседа
    matches = matcher.knnMatch(des1, des2, k=2)

    # Применяем тест отношения Лоу
    good_matches = []
    for m_n in matches:
        if len(m_n) == 2:
            m, n = m_n
            if m.distance < ratio * n.distance:
                good_matches.append(m)

    return good_matches

# Сопоставляем пары изображений
matches_list = []

for i in range(len(images) - 1):
    matches = match_images(descriptors_list[i], descriptors_list[i+1])
    matches_list.append(matches)
    print(f"Соответствий между изображениями {i+1} и {i+2}: {len(matches)}")

# Визуализируем соответствия
fig, axes = plt.subplots(len(matches_list), 1, figsize=(16, 6*len(matches_list)))

if len(matches_list) == 1:
    axes = [axes]

for i, matches in enumerate(matches_list):
    img_matches = cv2.drawMatches(
        images[i], keypoints_list[i],
        images[i+1], keypoints_list[i+1],
        matches[:50], None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )

    axes[i].imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
    axes[i].set_title(f'Соответствия: Изображение {i+1} ↔ Изображение {i+2} (показано 50/{len(matches)})')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 2: Вычисление гомографий
# ===============================

print("\n📋 ЗАДАНИЕ 2: Вычисление гомографий между изображениями")

def compute_homography_with_ransac(kp1, kp2, matches):
    """Вычисляет гомографию по соответствиям"""

    if len(matches) < 4:
        print("⚠️ Недостаточно соответствий для вычисления гомографии")
        return None, None

    # Извлекаем координаты точек
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Вычисляем гомографию с RANSAC
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    inliers = mask.ravel().tolist()
    n_inliers = sum(inliers)

    print(f"   Гомография: {n_inliers}/{len(matches)} inliers ({n_inliers/len(matches)*100:.1f}%)")

    return H, mask

# Вычисляем гомографии для всех пар
homographies = []

for i, matches in enumerate(matches_list):
    print(f"Вычисляем гомографию {i+1}→{i+2}:")
    H, mask = compute_homography_with_ransac(
        keypoints_list[i],
        keypoints_list[i+1],
        matches
    )
    homographies.append(H)

    if H is not None:
        print(f"   Матрица гомографии:")
        print(H)

In [ ]:
# ===============================
# ЗАДАНИЕ 3: Простая панорама из 2 изображений
# ===============================

print("\n📋 ЗАДАНИЕ 3: Создание простой панорамы из 2 изображений")

def stitch_two_images(img1, img2, H):
    """Сшивает два изображения используя гомографию"""

    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    # Определяем углы второго изображения
    corners2 = np.float32([[0, 0], [w2, 0], [w2, h2], [0, h2]]).reshape(-1, 1, 2)

    # Трансформируем углы второго изображения в систему координат первого
    corners2_transformed = cv2.perspectiveTransform(corners2, H)

    # Объединяем углы обоих изображений
    corners1 = np.float32([[0, 0], [w1, 0], [w1, h1], [0, h1]]).reshape(-1, 1, 2)
    all_corners = np.concatenate((corners1, corners2_transformed), axis=0)

    # Находим границы результирующего изображения
    [x_min, y_min] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(all_corners.max(axis=0).ravel() + 0.5)

    # Корректируем гомографию для учёта отрицательного сдвига
    translation_dist = [-x_min, -y_min]
    H_translation = np.array([[1, 0, translation_dist[0]],
                              [0, 1, translation_dist[1]],
                              [0, 0, 1]])

    # Размер выходного изображения
    output_width = x_max - x_min
    output_height = y_max - y_min

    print(f"   Размер панорамы: {output_width}x{output_height}")
    print(f"   Сдвиг: ({translation_dist[0]}, {translation_dist[1]})")

    # Трансформируем второе изображение
    img2_warped = cv2.warpPerspective(img2, H_translation @ H, (output_width, output_height))

    # Помещаем первое изображение на холст
    img2_warped[translation_dist[1]:h1+translation_dist[1],
                translation_dist[0]:w1+translation_dist[0]] = img1

    return img2_warped

# Сшиваем первые два изображения
if homographies[0] is not None:
    panorama_2img = stitch_two_images(images[0], images[1], homographies[0])

    plt.figure(figsize=(16, 6))
    plt.imshow(cv2.cvtColor(panorama_2img, cv2.COLOR_BGR2RGB))
    plt.title('Панорама из 2 изображений (простое наложение)')
    plt.axis('off')
    plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 4: Панорама из 3+ изображений
# ===============================

print("\n📋 ЗАДАНИЕ 4: Создание панорамы из всех изображений")

def stitch_multiple_images(images, homographies, base_idx=0):
    """
    Сшивает несколько изображений в панораму

    Args:
        images: список изображений
        homographies: список гомографий между соседними изображениями
        base_idx: индекс базового изображения (обычно центральное)

    Returns:
        panorama: результирующая панорама
    """

    n_images = len(images)
    print(f"🔄 Сшиваем {n_images} изображений...")

    # Вычисляем составные гомографии относительно базового изображения
    # (базовое изображение имеет единичную гомографию)
    cumulative_H = [None] * n_images
    cumulative_H[base_idx] = np.eye(3)

    # Гомографии для изображений справа от базового
    for i in range(base_idx + 1, n_images):
        if cumulative_H[i-1] is not None and homographies[i-1] is not None:
            cumulative_H[i] = cumulative_H[i-1] @ homographies[i-1]

    # Гомографии для изображений слева от базового
    for i in range(base_idx - 1, -1, -1):
        if cumulative_H[i+1] is not None and homographies[i] is not None:
            # Обратная гомография
            H_inv = np.linalg.inv(homographies[i])
            cumulative_H[i] = cumulative_H[i+1] @ H_inv

    # Находим границы результирующей панорамы
    corners_all = []

    for i, img in enumerate(images):
        if cumulative_H[i] is None:
            continue

        h, w = img.shape[:2]
        corners = np.float32([[0, 0], [w, 0], [w, h], [0, h]]).reshape(-1, 1, 2)
        corners_transformed = cv2.perspectiveTransform(corners, cumulative_H[i])
        corners_all.append(corners_transformed)

    corners_all = np.concatenate(corners_all, axis=0)

    [x_min, y_min] = np.int32(corners_all.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(corners_all.max(axis=0).ravel() + 0.5)

    # Корректируем гомографии для учёта отрицательного сдвига
    translation_dist = [-x_min, -y_min]
    H_translation = np.array([[1, 0, translation_dist[0]],
                              [0, 1, translation_dist[1]],
                              [0, 0, 1]])

    output_width = x_max - x_min
    output_height = y_max - y_min

    print(f"   Размер панорамы: {output_width}x{output_height}")

    # Создаём результирующее изображение
    panorama = np.zeros((output_height, output_width, 3), dtype=np.uint8)

    # Трансформируем и размещаем каждое изображение
    for i, img in enumerate(images):
        if cumulative_H[i] is None:
            print(f"   ⚠️ Пропускаем изображение {i+1} (нет гомографии)")
            continue

        H_adjusted = H_translation @ cumulative_H[i]
        img_warped = cv2.warpPerspective(img, H_adjusted, (output_width, output_height))

        # Простое наложение (последнее изображение перекрывает предыдущие)
        mask = (img_warped > 0).any(axis=2)
        panorama[mask] = img_warped[mask]

        print(f"   ✓ Добавлено изображение {i+1}")

    return panorama

# Выбираем центральное изображение как базовое
base_idx = len(images) // 2

panorama_full = stitch_multiple_images(images, homographies, base_idx=base_idx)

plt.figure(figsize=(20, 8))
plt.imshow(cv2.cvtColor(panorama_full, cv2.COLOR_BGR2RGB))
plt.title(f'Панорама из {len(images)} изображений (базовое: изображение {base_idx+1})')
plt.axis('off')
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 5: Улучшение качества - Alpha Blending
# ===============================

print("\n📋 ЗАДАНИЕ 5: Улучшение качества с помощью смешивания")

def stitch_with_blending(images, homographies, base_idx=0):
    """Сшивает изображения с alpha blending в областях перекрытия"""

    n_images = len(images)

    # Вычисляем составные гомографии (как раньше)
    cumulative_H = [None] * n_images
    cumulative_H[base_idx] = np.eye(3)

    for i in range(base_idx + 1, n_images):
        if cumulative_H[i-1] is not None and homographies[i-1] is not None:
            cumulative_H[i] = cumulative_H[i-1] @ homographies[i-1]

    for i in range(base_idx - 1, -1, -1):
        if cumulative_H[i+1] is not None and homographies[i] is not None:
            H_inv = np.linalg.inv(homographies[i])
            cumulative_H[i] = cumulative_H[i+1] @ H_inv

    # Находим границы
    corners_all = []
    for i, img in enumerate(images):
        if cumulative_H[i] is None:
            continue
        h, w = img.shape[:2]
        corners = np.float32([[0, 0], [w, 0], [w, h], [0, h]]).reshape(-1, 1, 2)
        corners_transformed = cv2.perspectiveTransform(corners, cumulative_H[i])
        corners_all.append(corners_transformed)

    corners_all = np.concatenate(corners_all, axis=0)
    [x_min, y_min] = np.int32(corners_all.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(corners_all.max(axis=0).ravel() + 0.5)

    translation_dist = [-x_min, -y_min]
    H_translation = np.array([[1, 0, translation_dist[0]],
                              [0, 1, translation_dist[1]],
                              [0, 0, 1]])

    output_width = x_max - x_min
    output_height = y_max - y_min

    # Создаём результирующее изображение и канал весов
    panorama = np.zeros((output_height, output_width, 3), dtype=np.float32)
    weights = np.zeros((output_height, output_height), dtype=np.float32)

    # Трансформируем и накапливаем изображения
    for i, img in enumerate(images):
        if cumulative_H[i] is None:
            continue

        H_adjusted = H_translation @ cumulative_H[i]

        # Трансформируем изображение
        img_warped = cv2.warpPerspective(img, H_adjusted, (output_width, output_height))

        # Создаём маску весов (расстояние до границы)
        h, w = img.shape[:2]
        mask = np.ones((h, w), dtype=np.float32)

        # Растушёвка по краям
        feather_width = 50
        for y in range(h):
            for x in range(w):
                dist_to_edge = min(x, w-x, y, h-y)
                if dist_to_edge < feather_width:
                    mask[y, x] = dist_to_edge / feather_width

        # Трансформируем маску
        mask_warped = cv2.warpPerspective(mask, H_adjusted, (output_width, output_height))

        # Накапливаем изображение и веса
        for c in range(3):
            panorama[:, :, c] += img_warped[:, :, c] * mask_warped
        weights += mask_warped

    # Нормализуем по весам
    for c in range(3):
        panorama[:, :, c] = np.divide(
            panorama[:, :, c],
            weights,
            out=np.zeros_like(panorama[:, :, c]),
            where=weights!=0
        )

    panorama = np.clip(panorama, 0, 255).astype(np.uint8)

    return panorama

print("🔄 Создаём панораму с alpha blending...")
panorama_blended = stitch_with_blending(images, homographies, base_idx=base_idx)

# Сравнение результатов
fig, axes = plt.subplots(2, 1, figsize=(20, 12))

axes[0].imshow(cv2.cvtColor(panorama_full, cv2.COLOR_BGR2RGB))
axes[0].set_title('Простое наложение (видны швы)')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(panorama_blended, cv2.COLOR_BGR2RGB))
axes[1].set_title('С alpha blending (швы размыты)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 6: Обрезка чёрных границ
# ===============================

print("\n📋 ЗАДАНИЕ 6: Автоматическая обрезка чёрных границ")

def crop_black_borders(panorama, threshold=5):
    """Обрезает чёрные границы панорамы"""

    # Конвертируем в grayscale
    gray = cv2.cvtColor(panorama, cv2.COLOR_BGR2GRAY)

    # Находим все ненулевые пиксели
    _, thresh = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

    # Находим контуры
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return panorama

    # Находим наибольший контур
    largest_contour = max(contours, key=cv2.contourArea)

    # Находим ограничивающий прямоугольник
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Обрезаем
    cropped = panorama[y:y+h, x:x+w]

    print(f"   Обрезка: ({x}, {y}) → ({x+w}, {y+h})")
    print(f"   Размер до обрезки: {panorama.shape[1]}x{panorama.shape[0]}")
    print(f"   Размер после обрезки: {w}x{h}")

    return cropped

panorama_cropped = crop_black_borders(panorama_blended)

plt.figure(figsize=(20, 6))
plt.imshow(cv2.cvtColor(panorama_cropped, cv2.COLOR_BGR2RGB))
plt.title('Финальная панорама (с обрезкой)')
plt.axis('off')
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 7: Сравнение с OpenCV Stitcher
# ===============================

print("\n📋 ЗАДАНИЕ 7: Сравнение с встроенным OpenCV Stitcher")

try:
    # Создаём stitcher
    stitcher = cv2.Stitcher_create()

    # Сшиваем изображения
    print("🔄 Применяем cv2.Stitcher...")
    status, panorama_opencv = stitcher.stitch(images)

    if status == cv2.Stitcher_OK:
        print("✅ OpenCV Stitcher: успешно!")

        # Сравнение
        fig, axes = plt.subplots(2, 1, figsize=(20, 12))

        axes[0].imshow(cv2.cvtColor(panorama_cropped, cv2.COLOR_BGR2RGB))
        axes[0].set_title('Наша реализация')
        axes[0].axis('off')

        axes[1].imshow(cv2.cvtColor(panorama_opencv, cv2.COLOR_BGR2RGB))
        axes[1].set_title('OpenCV Stitcher (с bundle adjustment и multiband blending)')
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()
    else:
        print(f"❌ OpenCV Stitcher: ошибка (код {status})")

except Exception as e:
    print(f"❌ Не удалось использовать cv2.Stitcher: {e}")

# ДОМАШНЕЕ ЗАДАНИЕ

1. Создайте 360° панораму:
- Сфотографируйте 8-12 изображений, вращаясь вокруг оси
- Сшейте их в полную круговую панораму
- Реализуйте цилиндрическую или сферическую проекцию

2. Автоматическая обрезка с минимизацией потерь:
- Реализуйте алгоритм, который находит максимальный прямоугольник
- Без чёрных пикселей внутри панорамы

3. Вертикальная панорама:
- Создайте вертикальную панораму (например, высокого здания)
- Адаптируйте алгоритм сшивки для вертикальной ориентации
